In [ ]:
import requests
from urllib.parse import urlparse, quote
import pandas as pd

In [ ]:

key_fd = open('roadapikey.txt', mode='r')
roadapi_key = key_fd.read(100)
key_fd.close()

In [ ]:
bldg_list = ['대구시청', '강원도청', '경북도청']
road_addr_list = []
for bldg in bldg_list:
    url = f'{road_url}?confmKey={roadapi_key}&currentPage=1&countPerPage=10&keyword={quote(bldg)}&resultType=json'
    results = requests.get(url).json()
    addr_list = results['results']['juso']
    road_addr = addr_list[0]['roadAddr']
    road_addr_list.append(road_addr)
    print(road_addr)

In [ ]:
import pandas as pd 
df = pd.DataFrame({'place':bldg_list, 'addr':road_addr_list})
df

In [ ]:
key_fd = open('kakaomapkey.txt', mode='r')
kmap_key = key_fd.read(100)
key_fd.close()

In [ ]:
lat_list = []; lng_list = []
for i in df.index:
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + quote(df['addr'][i])
    result = requests.get(url,
            headers={"Authorization": "KakaoAK "+kmap_key}).json()
    lat = float(result['documents'][0]['y'])
    lng = float(result['documents'][0]['x'])
    lat_list.append(lat)
    lng_list.append(lng)

In [ ]:
df['lat'] = lat_list
df['lng'] = lng_list
df

In [ ]:
df.lat.mean(), df.lng.mean()

In [ ]:
import folium

In [ ]:
map = folium.Map(location=[df.lat.mean(), df.lng.mean()], zoom_start=8)
folium.Marker(
    location=[36.575713, 128.471807],
    popup='경북도청'
).add_to(map)
map

In [ ]:
Make this Notebook Trusted to load map: File -> Trust Notebook


In [ ]:

map = folium.Map(location=[df.lat.mean(), df.lng.mean()], zoom_start=8)
for i in df.index:
    folium.Marker(
        location=[df.lat[i], df.lng[i]],
        popup=df.place[i],
    ).add_to(map)
title_html = '<h3 align="center" style="font-size:20px"><b>대구시청, 강원도청, 경북도청</b></h3>'   
map.get_root().html.add_child(folium.Element(title_html))
map

In [ ]:
Make this Notebook Trusted to load map: File -> Trust Notebook
